In [ ]:
# This note book contains the code for performing iteravtive pruning on trained Resnet18 model on CIFAR 10 dataset
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/compsci532/cnn-pruning-status_200/

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/compsci532/cnn-pruning-status_200


In [ ]:
import torch
import copy
import torch.nn.utils.prune as prune
import math
import random


In [ ]:
# Load the pretrained model
from cifar10_models.resnet import resnet18
my_model = resnet18()

# Pretrained model
my_model = resnet18(pretrained=True)
my_model.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [ ]:
#calculate sparsity ratio
def calculate_sparsity(model):
  total_count = 0
  zero_count = 0
  
  for buffer_name, buffer in model.named_buffers():
    zero_count += torch.sum(buffer == 0).item()
    if zero_count>0:
      total_count += buffer.nelement()

  print("Total params: ", total_count)
  print("Zero params: ", zero_count)
  return (math.ceil(zero_count*100/total_count))

In [ ]:
# Build and save the iterative pruning model with 50 % sparsity ratio
model1 = copy.deepcopy(my_model)
parameters_to_prune =[]
for module_name, module in model1.named_modules():
        if isinstance(module, torch.nn.Conv2d) or isinstance(module, torch.nn.Linear):
            parameters_to_prune.append((module, "weight"))
print("Layers to prune {}".format((len(parameters_to_prune))))
for i in range(5):
  prune.global_unstructured(
  parameters_to_prune,
  pruning_method=prune.L1Unstructured,
  amount=0.128,
  )
print("Sparsity of model1", calculate_sparsity(model1))
#save the model
torch.save(model1, "cifar10_models/state_dicts/iterative_50.pt")

Layers to prune 21
Total params:  11173972
Zero params:  5535550
Sparsity of model1 50


In [ ]:
# Build and save the iterative pruning model with 75 % sparsity ratio
model2 = copy.deepcopy(my_model)
parameters_to_prune =[]
for module_name, module in model2.named_modules():
        if isinstance(module, torch.nn.Conv2d) or isinstance(module, torch.nn.Linear):
            parameters_to_prune.append((module, "weight"))
print("Layers to prune {}".format((len(parameters_to_prune))))
for i in range(5):
  prune.global_unstructured(
  parameters_to_prune,
  pruning_method=prune.L1Unstructured,
  amount=0.24,
  )
print("Sparsity of model2", calculate_sparsity(model2))
#save the model
torch.save(model2, "cifar10_models/state_dicts/iterative_75.pt")


Layers to prune 21
Total params:  11173972
Zero params:  8333602
Sparsity of model2 75


In [ ]:
# Build and save the iterative pruning model with 90 % sparsity ratio
model3 = copy.deepcopy(my_model)
parameters_to_prune =[]
for module_name, module in model3.named_modules():
        if isinstance(module, torch.nn.Conv2d) or isinstance(module, torch.nn.Linear):
            parameters_to_prune.append((module, "weight"))
print("Layers to prune {}".format((len(parameters_to_prune))))
for i in range(5):
  prune.global_unstructured(
  parameters_to_prune,
  pruning_method=prune.L1Unstructured,
  amount=0.36,
  )
print("Sparsity of model3", calculate_sparsity(model3))
#save the model
torch.save(model3, "cifar10_models/state_dicts/iterative_90.pt")

Layers to prune 21
Total params:  11173972
Zero params:  9965588
Sparsity of model3 90
